In [1]:
import numpy
import pandas as pd 
import nltk
import string
import re
from nltk.corpus import stopwords
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
art_df = pd.read_csv('C:/Users/Powerhouse/Documents/GitHub/Project-High/article-database.csv')
print(art_df.shape)
art_df.head()

(2288, 6)


,Unnamed: 0,Name,Url,Text,Tags,Claps
0,0,None,https://medium.com/@elektra.sunn/cheshire-moon...,NaN,[],[]
1,0,None,https://medium.com/@nicolascole77/if-youre-doi...,NaN,[],[]
2,0,The Ultimate Travel Workout and Diet Plans by ...,https://medium.com/@garrettgee/the-bucket-list...,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac...",[]
3,0,Building a Responsive Image Gallery,https://medium.com/backticks-tildes/building-a...,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']",[]
4,0,When they’re already “forever” for you,https://medium.com/@krisgage/when-theyre-alrea...,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo...",[]


In [3]:
model_db = art_df.drop([art_df.columns[0], art_df.columns[1], art_df.columns[2], art_df.columns[5]], axis=1)
model_db.head()

,Text,Tags
0,NaN,[]
1,NaN,[]
2,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac..."
3,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']"
4,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo..."


In [4]:
model_db.isnull().sum()

Text    119
Tags      0
dtype: int64

In [5]:
model_db_clean = model_db.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False).reset_index(drop=True)
model_db_clean.head()

,Text,Tags
0,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac..."
1,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']"
2,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo..."
3,by Jean-Louis Gassée [A delayed Monday Note d...,"['Publishing', 'Apple', 'Apple']"
4,"Today, I was integrating FCM in my iOS app. I...","['Swift', 'iOS', 'Apple']"


In [6]:
# from ast import literal_eval
def func(raw_tags):
    raw_split = raw_tags[1:-1].split(',')
    num_tags = len(raw_split)
    tags_clean = []
    tags_clean.append(raw_split[0][1:-1])
    for i in range(1, num_tags):
       tags_clean.append(raw_split[i][2:-1])
    return tags_clean

model_db_clean['Tags_clean'] = model_db_clean['Tags'].apply(lambda x: func(x))

# model_db_clean.loc[:,'Tags'] = model_db_clean.loc[:,'Tags'].apply(lambda x: literal_eval(x))
print(type(model_db_clean['Tags']))
model_db_clean.head()

<class 'pandas.core.series.Series'>


,Text,Tags,Tags_clean
0,"INTRODUCING THE BUCKET LIST FAMILY Hi, we are...","['Fitness', 'Travel', 'Diet', 'Exercise', 'Vac...","[Fitness, Travel, Diet, Exercise, Vacation]"
1,In this post you would learn to build a basic...,"['Design', 'Data', 'Growth', 'CSS', 'Web Design']","[Design, Data, Growth, CSS, Web Design]"
2,Because love that’s “light” can last that lon...,"['Love', 'Relationships', 'Dating', 'Men', 'Wo...","[Love, Relationships, Dating, Men, Women]"
3,by Jean-Louis Gassée [A delayed Monday Note d...,"['Publishing', 'Apple', 'Apple']","[Publishing, Apple, Apple]"
4,"Today, I was integrating FCM in my iOS app. I...","['Swift', 'iOS', 'Apple']","[Swift, iOS, Apple]"


In [7]:
multi_label_transform = MultiLabelBinarizer()
multi_label_transform.fit(model_db_clean.Tags_clean)

y = multi_label_transform.transform(model_db_clean.Tags_clean)
print(y.shape, model_db_clean.shape)

(2169, 684) (2169, 3)


In [8]:
cols = []
for i in list(multi_label_transform.classes_):     
    cols.append(i)
cols.append('Text')

prepd_db = pd.DataFrame()

prepd_db.loc[:, 'Text'] = model_db_clean.loc[:, 'Text']
for i in range(0, y.shape[1]):
        prepd_db.loc[:, i+1] = y[:, i]

def clean_text(text):
    text = re.sub("\'", "", text) 
    text = re.sub("[^a-zA-Z]"," ",text) 
    text = ' '.join(text.split()) 
    text = text.lower() 
    
    return text

prepd_db['Text_clean'] = prepd_db['Text'].apply(lambda x: clean_text(x))
prepd_db.drop(['Text'], axis=1, inplace=True)

prepd_db.columns = cols
prepd_db.drop([''], axis=1, inplace=True)

prepd_db.head()

,2016 Election,2017,AI,API,AWS,Abortion,Abuse,Academia,Accessibility,Activism,...,Writing,Writing Prompts,Writing Tips,Yoga,YouTube,iOS,iOS App Development,iPad,iPhone,Text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,introducing the bucket list family hi we are t...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,in this post you would learn to build a basic ...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,because love that s light can last that long a...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,by jean louis gass e a delayed monday note due...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,today i was integrating fcm in my ios app i sa...


In [9]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

prepd_db['Text'] = prepd_db['Text'].apply(lambda x: remove_stopwords(x))
prepd_db.head()

,2016 Election,2017,AI,API,AWS,Abortion,Abuse,Academia,Accessibility,Activism,...,Writing,Writing Prompts,Writing Tips,Yoga,YouTube,iOS,iOS App Development,iPad,iPhone,Text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,introducing bucket list family hi bucket list ...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,post would learn build basic image gallery fle...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,love light last long past partners frankly pro...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,jean louis gass e delayed monday note due logi...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,today integrating fcm ios app saw word method ...


In [10]:
prepd_db.to_csv('model-data.csv')

In [11]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=1000)

xtrain, xval, ytrain, yval = train_test_split(prepd_db['Text'], y, test_size=0.2, random_state=9)

xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

clf.fit(xtrain_tfidf, ytrain)
y_pred = clf.predict(xval_tfidf)

multi_label_transform.inverse_transform(y_pred)[0]

C:\Program Files\Python37\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
C:\Program Files\Python37\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 37 is present in all training examples.
  str(classes[c]))
C:\Program Files\Python37\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 44 is present in all training examples.
  str(classes[c]))
C:\Program Files\Python37\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 85 is present in all training examples.
  str(classes[c]))
C:\Program Files\Python37\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 113 is present in all training examples.
  str(classes[c]))
C:\Program Files\Python37\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 126 is present in all training examples.
  str(classes[c]))
C:\Program Files\Python37\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Labe

()